# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,-23.34,99,68,1.90,RU,1733160297
1,1,hawaiian paradise park,19.5933,-154.9731,19.94,95,4,3.51,US,1733160298
2,2,tupiza,-21.4500,-65.7167,22.26,31,100,4.45,BO,1733160044
3,3,jamestown,42.0970,-79.2353,-1.10,95,100,4.63,US,1733160169
4,4,ushuaia,-54.8000,-68.3000,7.81,81,75,7.20,AR,1733160301


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:


# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    "Lat",
    geo = True,
    tiles = 'OSM',
    frame_width=1000,
    frame_height=600,
    size = 'Humidity',
    color = 'City')


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 22) & 
    (city_data_df['Humidity'] < 80) & 
    (city_data_df['Cloudiness'] < 40) & 
    (city_data_df['Wind Speed'] < 10)
].dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,salinopolis,-0.6136,-47.3561,29.56,67,25,4.82,BR,1733160305
10,10,saint-pierre,-21.3393,55.4781,25.82,69,0,1.03,RE,1733160308
16,16,lazaro cardenas,17.9583,-102.2000,28.20,58,0,1.43,MX,1733160315
31,31,mocimboa,-11.3167,40.3500,25.32,70,23,2.40,MZ,1733160331
39,39,wailua homesteads,22.0669,-159.3780,22.44,78,0,3.09,US,1733160340
...,...,...,...,...,...,...,...,...,...,...
535,535,yola,9.2000,12.4833,31.90,33,0,2.06,NG,1733160897
538,538,mandiana,10.6333,-8.6833,28.11,21,5,1.64,GN,1733160900
546,546,numan,9.4667,12.0333,25.80,26,27,1.76,NG,1733160909
565,565,yeppoon,-23.1333,150.7333,24.66,79,38,4.81,AU,1733160931


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,salinopolis,BR,-0.6136,-47.3561,67,
10,saint-pierre,RE,-21.3393,55.4781,69,
16,lazaro cardenas,MX,17.9583,-102.2000,58,
31,mocimboa,MZ,-11.3167,40.3500,70,
39,wailua homesteads,US,22.0669,-159.3780,78,
...,...,...,...,...,...,...
535,yola,NG,9.2000,12.4833,33,
538,mandiana,GN,10.6333,-8.6833,21,
546,numan,NG,9.4667,12.0333,26,
565,yeppoon,AU,-23.1333,150.7333,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
salinopolis - nearest hotel: Hotel Salinópolis
saint-pierre - nearest hotel: Tropic Hotel
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
mocimboa - nearest hotel: Vumba
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
dzilam gonzalez - nearest hotel: No hotel found
beau vallon - nearest hotel: No hotel found
jiwani - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
chamical - nearest hotel: No hotel found
isla mujeres - nearest hotel: Villa La Bella
lihue - nearest hotel: Kauai Palms
vila velha - nearest hotel: Hotel Prainha
port sudan - nearest hotel: Ranya Hotel
bereeda - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
kingston - nearest hotel: Aphrodite Hotel
bouar - nearest hotel: Mission catholique
bopolu - nearest hotel: No hotel found
las khorey - nearest hotel: No hotel found
bata - nearest hotel: Elik Melen
caluula - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
7,salinopolis,BR,-0.6136,-47.3561,67,Hotel Salinópolis
10,saint-pierre,RE,-21.3393,55.4781,69,Tropic Hotel
16,lazaro cardenas,MX,17.9583,-102.2000,58,Hotel Sol del Pacífico
31,mocimboa,MZ,-11.3167,40.3500,70,Vumba
39,wailua homesteads,US,22.0669,-159.3780,78,Hilton Garden Inn Kauai Wailua Bay
...,...,...,...,...,...,...
535,yola,NG,9.2000,12.4833,33,No hotel found
538,mandiana,GN,10.6333,-8.6833,21,Hôtel Prince
546,numan,NG,9.4667,12.0333,26,No hotel found
565,yeppoon,AU,-23.1333,150.7333,79,Blossom's on Seaspray


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot_3 = hotel_df.hvplot.points(
    'Lng',
    "Lat",
    geo = True,
    tiles = 'OSM',
    frame_width=1000,
    frame_height=600,
    size = 'Humidity',
    color = 'City',
    hover_cols=['City', 'Hotel Name', 'Country'])

# Display the map
map_plot_3

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)